In [1]:
import import_ipynb
from dcgan import DCGAN
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import fashion_mnist
from sklearn.utils import shuffle
from imutils import build_montages
import numpy as np
import cv2
import os

importing Jupyter notebook from dcgan.ipynb


Loading MNIST

In [2]:
print("loading MNIST dataset...")
((trainX, _), (testX, _)) = fashion_mnist.load_data()
trainImages = np.concatenate([trainX, testX])
print(trainImages.shape)
trainImages = np.expand_dims(trainImages, axis=-1)
print(trainImages.shape)
trainImages = (trainImages.astype("float") - 127.5) / 127.5
print(trainImages.shape)

loading MNIST dataset...
(70000, 28, 28)
(70000, 28, 28, 1)
(70000, 28, 28, 1)


__Setting hyper parameters__

In [3]:
NUM_EPOCHS = 50
BATCH_SIZE = 128
INIT_LR = 2e-4

__Building generator and then \
Building discriminator and then \
Compiling Discriminator__

In [4]:
gen = DCGAN.build_generator(7, 64, channels=1)
gen.summary()
disc = DCGAN.build_discriminator(28, 28, 1)
disc.summary()
discOpt = Adam(lr=INIT_LR, beta_1=0.5, decay=INIT_LR / NUM_EPOCHS)
disc.compile(loss="binary_crossentropy", optimizer=discOpt)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               51712     
_________________________________________________________________
activation (Activation)      (None, 512)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 512)               2048      
_________________________________________________________________
dense_1 (Dense)              (None, 3136)              1608768   
_________________________________________________________________
activation_1 (Activation)    (None, 3136)              0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 3136)              12544     
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 64)          0

__Building and compiling GAN (without discriminator part)__ \
build the adversarial model by first setting the discriminator to \
*NOT* be trainable, then combine the generator and discriminator together

In [5]:
disc.trainable = False  # because it is already compiled in above cell
ganInput = Input(shape=(100,))
gen_out = gen(ganInput)
ganOutput = disc(gen_out)
gan = Model(ganInput, ganOutput)  #FINAL COMBINED MODEL
gan.summary()
ganOpt = Adam(lr=INIT_LR, beta_1=0.5, decay=INIT_LR / NUM_EPOCHS) 
gan.compile(loss="binary_crossentropy", optimizer=discOpt)   #COMPILING THE GAN

__TRAINING OF GAN__

In [8]:
benchmarkNoise = np.random.uniform(-1, 1, size=(256, 100))
# loop over the epochs
for epoch in range(0, NUM_EPOCHS):
    print("[INFO] starting epoch {} of {}...".format(epoch + 1,NUM_EPOCHS))
    batchesPerEpoch = int(trainImages.shape[0] / BATCH_SIZE)
    for i in range(0, batchesPerEpoch):
        p = None
        imageBatch = trainImages[i * BATCH_SIZE:(i + 1) * BATCH_SIZE]
        noise = np.random.uniform(-1, 1, size=(BATCH_SIZE, 100))
        genImages = gen.predict(noise, verbose=0)
        X = np.concatenate((imageBatch, genImages))
        y = ([1] * BATCH_SIZE) + ([0] * BATCH_SIZE)
        y = np.reshape(y, (-1,))
        #Till now X consisted of 'BATCH_SIZE' real images and 'BATCH_SIZE' fake images
        #and till now y consisted of 'BATCH_SIZE' '1' labels and 'BATCH_SIZE' '0' labels
        (X, y) = shuffle(X, y)  #shuffling the X and y in a similar fashin so that labels remain correct
        # train ONLY the discriminator on the data (not the entire GAN model)
        discLoss = disc.train_on_batch(X, y)
        
        
        #After training the disc on a batch, we will freeze disc's weights and try to fool it using fake labels
        #so that we can train our gen
        
        # Now training our generator via the adversarial model by
        # (1) generating random noise and (2) training the generator
        # WITH DISC WEIGHTS FROZEN
        noise = np.random.uniform(-1, 1, (BATCH_SIZE, 100))
        fakeLabels = [1] * BATCH_SIZE
        fakeLabels = np.reshape(fakeLabels, (-1,))
        ganLoss = gan.train_on_batch(noise, fakeLabels)
        
        
        
        # check to see if this is the end of an epoch, and if so,
        # initialize the output path
        if i == batchesPerEpoch - 1:
            p = ['output', "epoch_{}_output.png".format(str(epoch + 1).zfill(4))]
        else:
            if epoch < 10 and i % 25 == 0:
                p = ["output", "epoch_{}_step_{}.png".format(
                    str(epoch + 1).zfill(4), str(i).zfill(5))]
            # visualizations later in the training process are less
            # interesting
            elif epoch >= 10 and i % 100 == 0:
                p = ["output", "epoch_{}_step_{}.png".format(
                    str(epoch + 1).zfill(4), str(i).zfill(5))]
        
        
        
        if p is not None:
            # show loss information
            print("[INFO] Step {}_{}: discriminator_loss={:.6f}, ""adversarial_loss={:.6f}".format(epoch + 1, i,discLoss, ganLoss))
            images = gen.predict(benchmarkNoise)
            images = ((images * 127.5) + 127.5).astype("uint8")
            images = np.repeat(images, 3, axis=-1)
            vis = build_montages(images, (28, 28), (16, 16))[0]
            # write the visualization to disk
            p = os.path.sep.join(p)
            cv2.imwrite(p, vis)

[INFO] starting epoch 1 of 50...
[INFO] Step 1_0: discriminator_loss=0.583581, adversarial_loss=0.496276
[INFO] Step 1_25: discriminator_loss=0.028674, adversarial_loss=0.007920
[INFO] Step 1_50: discriminator_loss=0.000400, adversarial_loss=0.000859
[INFO] Step 1_75: discriminator_loss=0.000073, adversarial_loss=0.000470
[INFO] Step 1_100: discriminator_loss=0.000183, adversarial_loss=0.000296
[INFO] Step 1_125: discriminator_loss=0.000053, adversarial_loss=0.000229
[INFO] Step 1_150: discriminator_loss=0.000088, adversarial_loss=0.000197
[INFO] Step 1_175: discriminator_loss=0.000101, adversarial_loss=0.000202
[INFO] Step 1_200: discriminator_loss=0.000213, adversarial_loss=0.000259
[INFO] Step 1_225: discriminator_loss=0.014193, adversarial_loss=2.096987
[INFO] Step 1_250: discriminator_loss=0.136351, adversarial_loss=0.913766
[INFO] Step 1_275: discriminator_loss=0.137335, adversarial_loss=1.256148
[INFO] Step 1_300: discriminator_loss=0.196647, adversarial_loss=2.193867
[INFO] Ste

[INFO] Step 5_425: discriminator_loss=0.585067, adversarial_loss=0.820794
[INFO] Step 5_450: discriminator_loss=0.600117, adversarial_loss=1.036664
[INFO] Step 5_475: discriminator_loss=0.585143, adversarial_loss=0.801277
[INFO] Step 5_500: discriminator_loss=0.603626, adversarial_loss=1.140142
[INFO] Step 5_525: discriminator_loss=0.600796, adversarial_loss=0.995953
[INFO] Step 5_545: discriminator_loss=0.562358, adversarial_loss=1.027796
[INFO] starting epoch 6 of 50...
[INFO] Step 6_0: discriminator_loss=0.584364, adversarial_loss=0.850607
[INFO] Step 6_25: discriminator_loss=0.595024, adversarial_loss=0.984172
[INFO] Step 6_50: discriminator_loss=0.604630, adversarial_loss=1.274780
[INFO] Step 6_75: discriminator_loss=0.567475, adversarial_loss=0.963754
[INFO] Step 6_100: discriminator_loss=0.575165, adversarial_loss=0.707641
[INFO] Step 6_125: discriminator_loss=0.631713, adversarial_loss=0.616189
[INFO] Step 6_150: discriminator_loss=0.604316, adversarial_loss=1.003188
[INFO] Ste

[INFO] Step 10_275: discriminator_loss=0.653407, adversarial_loss=0.674248
[INFO] Step 10_300: discriminator_loss=0.613956, adversarial_loss=0.855443
[INFO] Step 10_325: discriminator_loss=0.630329, adversarial_loss=0.833934
[INFO] Step 10_350: discriminator_loss=0.577358, adversarial_loss=1.081311
[INFO] Step 10_375: discriminator_loss=0.573952, adversarial_loss=0.852742
[INFO] Step 10_400: discriminator_loss=0.630629, adversarial_loss=0.964596
[INFO] Step 10_425: discriminator_loss=0.648435, adversarial_loss=0.681091
[INFO] Step 10_450: discriminator_loss=0.623823, adversarial_loss=0.940317
[INFO] Step 10_475: discriminator_loss=0.630440, adversarial_loss=0.971541
[INFO] Step 10_500: discriminator_loss=0.626965, adversarial_loss=1.025918
[INFO] Step 10_525: discriminator_loss=0.598328, adversarial_loss=0.928919
[INFO] Step 10_545: discriminator_loss=0.616117, adversarial_loss=1.166975
[INFO] starting epoch 11 of 50...
[INFO] Step 11_0: discriminator_loss=0.629903, adversarial_loss=0.

[INFO] Step 24_100: discriminator_loss=0.585850, adversarial_loss=1.029275
[INFO] Step 24_200: discriminator_loss=0.564712, adversarial_loss=0.956214
[INFO] Step 24_300: discriminator_loss=0.575644, adversarial_loss=1.011854
[INFO] Step 24_400: discriminator_loss=0.629178, adversarial_loss=0.887247
[INFO] Step 24_500: discriminator_loss=0.573016, adversarial_loss=1.025619
[INFO] Step 24_545: discriminator_loss=0.559605, adversarial_loss=1.242487
[INFO] starting epoch 25 of 50...
[INFO] Step 25_0: discriminator_loss=0.596493, adversarial_loss=0.922956
[INFO] Step 25_100: discriminator_loss=0.565112, adversarial_loss=1.087754
[INFO] Step 25_200: discriminator_loss=0.557693, adversarial_loss=1.044543
[INFO] Step 25_300: discriminator_loss=0.572100, adversarial_loss=0.874089
[INFO] Step 25_400: discriminator_loss=0.637922, adversarial_loss=1.049267
[INFO] Step 25_500: discriminator_loss=0.597751, adversarial_loss=1.101028
[INFO] Step 25_545: discriminator_loss=0.578233, adversarial_loss=1.

[INFO] Step 39_100: discriminator_loss=0.604287, adversarial_loss=1.773386
[INFO] Step 39_200: discriminator_loss=0.476131, adversarial_loss=1.125995
[INFO] Step 39_300: discriminator_loss=0.518033, adversarial_loss=1.184954
[INFO] Step 39_400: discriminator_loss=0.583251, adversarial_loss=0.874747
[INFO] Step 39_500: discriminator_loss=0.537809, adversarial_loss=1.071350
[INFO] Step 39_545: discriminator_loss=0.529084, adversarial_loss=1.137412
[INFO] starting epoch 40 of 50...
[INFO] Step 40_0: discriminator_loss=0.518169, adversarial_loss=1.133886
[INFO] Step 40_100: discriminator_loss=0.541936, adversarial_loss=1.286594
[INFO] Step 40_200: discriminator_loss=0.462737, adversarial_loss=1.235438
[INFO] Step 40_300: discriminator_loss=0.523712, adversarial_loss=1.355727
[INFO] Step 40_400: discriminator_loss=0.606096, adversarial_loss=0.834406
[INFO] Step 40_500: discriminator_loss=0.543365, adversarial_loss=0.882985
[INFO] Step 40_545: discriminator_loss=0.552048, adversarial_loss=1.